<a href="https://colab.research.google.com/github/melrepa/MCD_IngCaracteristicas/blob/main/Tarea_3_Descargando_datos_en_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Descargando datos XML
Melissa Reyes Paz

<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/melrepa/MCD_IngCaracteristicas/blob/main/Tarea_3_Descargando_datos_en_XML.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

# Importando librerías necesarias para la descarga de XML.

In [2]:
import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
import requests # Otra forma no de las librerías de uso comun
import xml.etree.ElementTree as et #Implementa una simple y eficiente API para analizar y crear XML data.
import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import numpy as np
import zipfile # Descompresión de archivos
import re

# Archivos xml

Los archivos *xml* son una manera de compartir información a través de internet o de guardar información con formatos genéricos que sigue siendo muy utilizada hoy en día. En general lidiar con archivos xml es una pesadilla y se necesita explorarlos con calma y revisarlos bien antes de usarlos. 

La definición del formato y su uso de pede revisar en [este tutorial de la w3schools](https://www.w3schools.com/xml/default.asp).

Wikipedia es un buen ejemplo de un lugar donde la información se guarda y se descarga en forma de archivos xml. Por ejemplo, si queremos descargar datos de la wikipedia [con su herramienta de exportación](https://es.wikipedia.org/wiki/Especial:Exportar) utilizando [las categorias definidas por Wikipedia](https://es.wikipedia.org/wiki/Portal:Portada). Para hacerlo en forma programática es ecesario usar la [API de Mediawiki](https://github.com/mudroljub/wikipedia-api-docs) que veremos más adelante.

Por el momento descargemos unos datos de *wikipedia* y hagamos el ejercicio de tratar de entender la estructura del árbol. [-Waissman (2022)](https://github.com/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/descarga_datos.ipynb)

In [3]:
archivo_url = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml"
archivo_nombre = "poetas.xml"
subdir = "./data/"

if not os.path.exists(subdir + archivo_nombre):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(archivo_url, subdir + archivo_nombre)

poetas = et.parse(subdir + archivo_nombre)
root = poetas.getroot()

Los últimos códigos nos servirán para poder navegar entre los bloques de información del archivo XML.
Observando el arbol que se genera, y considerando que nunca había trabajado con un archivo XML, puedo decir qué a primera vista la información parece un poco confusa.

En primera instancia veo que se encuentra publicado por mediawiki. Después aparecen códigos que me imagino forman parte de la estructura del archivo o de la página como "Usuario", "Discusión", "Plantilla", "Ayuda", etc. 

A continuación aparece el nombre de una persona, una de las autoras, seguido de id, un timestamp y después un texto con la información general de la autora como su fecha y lugar de nacimiento. 
Todo está unido: enlaces, referencias, notas. Y así es para cada uno de las categorías del documento.

En pocas palabras, cada categoría o bloque de información, o atributo del archivo, contiene la información de poetas argentinos: su nombre, biografía, fechas de modificación del artículo, quien lo modificó.

Para poder acceder a diferentes bloques de información se tiene la estructura

> '{ http://www.mediawiki.org/xml/export-0.10/ }NombreDelAtributo'

In [9]:
[elem.tag for elem in root.iter()][50:60]

['{http://www.mediawiki.org/xml/export-0.10/}id',
 '{http://www.mediawiki.org/xml/export-0.10/}comment',
 '{http://www.mediawiki.org/xml/export-0.10/}model',
 '{http://www.mediawiki.org/xml/export-0.10/}format',
 '{http://www.mediawiki.org/xml/export-0.10/}text',
 '{http://www.mediawiki.org/xml/export-0.10/}sha1',
 '{http://www.mediawiki.org/xml/export-0.10/}page',
 '{http://www.mediawiki.org/xml/export-0.10/}title',
 '{http://www.mediawiki.org/xml/export-0.10/}ns',
 '{http://www.mediawiki.org/xml/export-0.10/}id']

A continuación, se accede a los bloques de información como el nombre del autor, biografía, fechas de modificación del artículo de wikipedia, el nombre del usuario que editó el artículo y esta información la ponemos en una tabla haciendo uso de pandas. 


In [12]:
author = []
for autor in root.iter('{http://www.mediawiki.org/xml/export-0.10/}title'):
    author.append(autor.text)

In [13]:
fechas_mod = []
for revision in root.iter('{http://www.mediawiki.org/xml/export-0.10/}timestamp'):
    fechas_mod.append(revision.text)

In [14]:
biogr = []
for bio in root.iter('{http://www.mediawiki.org/xml/export-0.10/}text'):
    biogr.append(bio.text)

In [15]:
usuario = []
for usname in root.iter('{http://www.mediawiki.org/xml/export-0.10/}username'):
    usuario.append(usname.text)

In [16]:
data = pd.DataFrame(columns = ['Autor','Biografia','Fecha_Modificacion'])
for i in range(len(author)):
  data = pd.concat([data,
                  pd.DataFrame.from_records([{'Autor': author[i], 
                                              'Biografia': biogr[i], 
                                              "Fecha_Modificacion": fechas_mod[i]}]
                                            )], ignore_index=True)

También se puede extraer mas información de la parte de la biografía. Como mencioné antes, incluye fecha de nacimiento. El bloque de información de la biografía es una cadena de caracteres y para hacer uso de esta información se necesita utilizar expresiones regulares. 


In [17]:
year = []
for i in range(len(data)):
  
  try:
    year.append(re.search(r"\[\[([1-3][0-9]{3})\]\]", data["Biografia"].iloc[i]).group(1))
  except:
    year.append(np.nan)

In [18]:
data["Año_Nacimiento"] = year

In [19]:
data

,Autor,Biografia,Fecha_Modificacion,Año_Nacimiento
0,Julia Morilla de Campbell,'''Julia Morilla de Campbell''' ([[Rosario (Ar...,2017-10-30T07:46:11Z,1922
1,Luis Negreti,{{Ficha de escritor\n|Imagen = NE...,2016-09-17T17:35:26Z,1890
2,Poldy Bird,{{Ficha de persona\n| padres = Enrique Bird Mo...,2018-06-11T04:34:41Z,1941
3,Ana María Shua,{{Ficha de persona\n|imagen=\n|nombre de nacim...,2018-06-13T18:50:17Z,1951
4,León Benarós,{{Ficha de persona\n|nombre = León B...,2018-04-29T21:49:56Z,1915
...,...,...,...,...
634,Humberto Tejera,{{Ficha de persona\n| nombre = Humberto Tejera...,2018-07-06T17:07:38Z,NaN
635,Mario Molina Cruz,{{Ficha de escritor\n|nombre= Mario Molina Cru...,2018-07-23T14:27:14Z,1955
636,Luis Ignacio Helguera,{{Ficha de persona\n| nombre = Luis Ignacio He...,2018-05-11T14:19:35Z,1962
637,Daniel Olivares Viniegra,{{Promocional|8|julio}}\n\n'''Daniel Olivares ...,2018-07-20T21:56:20Z,NaN


Una vez obtenida la información de una forma mas bonita o entendible a la vista de una persona que nunca había trabajado con XML (como yo) se pueden hacer y responder mas preguntas, por ejmeplo: 
* ¿Cuántas modificaciones del artículo se hicieron en cierto lapso? 
* ¿Dichas modificaciones fueron el resultado de algún evento en especial? 
* ¿Cuántos autores comparten año de nacimiento? 
* Si hay muchos autores que nacieron en el mismo año o en la misma generación, ¿cuál sería el factor social que impulso a estas personas a convertirse en poetas?
En esta última pregunta me refiero a que si de los 639 autores, por ejemplo, 80 nacieron en la década de 1920, ¿comparten una inspiración o una situación en común que los impulsara a ser poetas?

Hay muchas preguntas que se pueden realizar y responder incluso con solo tener el año de naciemiento de estos poetas.

El último paso a realizar es convertir el data frame elaborado como un archivo parquet.

In [20]:
data.to_parquet('poetas.parquet.gzip',compression='gzip')